In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# Define the dataset path
data_dir = 'Data'

# Parameters
img_height, img_width = 128, 128  # Resize images to 128x128
batch_size = 32

# Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,   # Normalize pixel values
    validation_split=0.2,  # 20% data for validation
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [5]:

# Load Training and Validation Data
train_data = data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 1710 images belonging to 9 classes.
Found 423 images belonging to 9 classes.


In [6]:
classes = 9

# Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu')
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(classes, activation='softmax')  # 9 output classes
])


/Users/balajikartheek/Desktop/Websites/Survailance2/signLanguage/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [6]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

/Users/balajikartheek/Desktop/Websites/Survailance2/signLanguage/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 25s 435ms/step - accuracy: 0.1821 - loss: 2.1767 - val_accuracy: 0.4610 - val_loss: 1.5274
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 22s 403ms/step - accuracy: 0.4599 - loss: 1.4670 - val_accuracy: 0.7329 - val_loss: 0.8796
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 21s 394ms/step - accuracy: 0.6041 - loss: 1.0271 - val_accuracy: 0.7991 - val_loss: 0.6240
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 474ms/step - accuracy: 0.6987 - loss: 0.8322 - val_accuracy: 0.8747 - val_loss: 0.4299
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 25s 456ms/step - accuracy: 0.7594 - loss: 0.6658 - val_accuracy: 0.8369 - val_loss: 0.4390


In [9]:

# Save the Model
# model.save('sign_language_model.h5')

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {test_accuracy * 100:.2f}%")


/Users/balajikartheek/Desktop/Websites/Survailance2/signLanguage/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 205ms/step - accuracy: 0.8541 - loss: 0.4319
Validation Accuracy: 86.52%


In [8]:
from tensorflow.keras.models import load_model
model = load_model('sign_language_model.h5')

In [10]:
# Testing
from tensorflow.keras.preprocessing import image

def predict_class(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions)
    class_label = list(train_data.class_indices.keys())[class_idx]
    return class_label

In [11]:

# Test Example
example_image = 'Data/Hello/Image_1729946405.997163.jpg'
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Predicted Class: Hello


In [12]:

# Test Example
example_image = 'Data/Victory/Image_1729947769.15842.jpg'
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Class: Victory
